# Building the 12-factor agent template from scratch in Python

Steps to start from a bare Python repo and build up a 12-factor agent. This walkthrough will guide you through creating a Python agent that follows the 12-factor methodology with BAML.

## Chapter 0 - Hello World

Let's start with a basic Python setup and a hello world program.

This guide will walk you through building agents in Python with BAML.

We'll start simple with a hello world program and gradually build up to a full agent.

For this notebook, you'll need to have your OpenAI API key saved in Google Colab secrets.


Here's our simple hello world program:

In [ ]:
# ./walkthrough/00-main.py
def hello():
    print('hello, world!')

def main():
    hello()

Let's run it to verify it works:

In [ ]:
main()

## Chapter 1 - CLI and Agent Loop

Now let's add BAML and create our first agent with a CLI interface.

In this chapter, we'll integrate BAML to create an AI agent that can respond to user input.

First, let's set up BAML support in our notebook.


### BAML Setup

Don't worry too much about this setup code - it will make sense later! For now, just know that:
- BAML is a tool for working with language models
- We need some special setup code to make it work nicely in Google Colab
- The `get_baml_client()` function will be used to interact with AI models

In [ ]:
!pip install baml-py

In [ ]:
import subprocess
from google.colab import userdata
import os

def baml_generate():
    try:
        result = subprocess.run(
            ["baml-cli", "generate"],
            check=True,
            capture_output=True,
            text=True
        )
        if result.stdout:
            print("[baml-cli generate]\n", result.stdout)
        if result.stderr:
            print("[baml-cli generate]\n", result.stderr)
    except subprocess.CalledProcessError as e:
        msg = (
            f"`baml-cli generate` failed with exit code {e.returncode}\n"
            f"--- STDOUT ---\n{e.stdout}\n"
            f"--- STDERR ---\n{e.stderr}"
        )
        raise RuntimeError(msg) from None

def get_baml_client():
    """
    a bunch of fun jank to work around the google colab import cache
    """
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    
    baml_generate()
    
    import importlib
    import baml_client
    importlib.reload(baml_client)
    return baml_client.sync_client.b


In [ ]:
!baml-cli init

Now let's create our agent that will use BAML to process user input.

First, we'll define the core agent logic:


In [ ]:
# ./walkthrough/01-agent.py
import json
from typing import Dict, Any, List

# tool call or a respond to human tool
AgentResponse = Any  # This will be the return type from b.DetermineNextStep

class Event:
    def __init__(self, type: str, data: Any):
        self.type = type
        self.data = data

class Thread:
    def __init__(self, events: List[Dict[str, Any]]):
        self.events = events
    
    def serialize_for_llm(self):
        # can change this to whatever custom serialization you want to do, XML, etc
        # e.g. https://github.com/got-agents/agents/blob/59ebbfa236fc376618f16ee08eb0f3bf7b698892/linear-assistant-ts/src/agent.ts#L66-L105
        return json.dumps(self.events)

# right now this just runs one turn with the LLM, but
# we'll update this function to handle all the agent logic
def agent_loop(thread: Thread) -> AgentResponse:
    b = get_baml_client()  # This will be defined by the BAML setup
    next_step = b.DetermineNextStep(thread.serialize_for_llm())
    return next_step

Next, we need to define the BAML function that our agent will use.

This BAML file defines what our agent can do:


In [ ]:
!curl -fsSL -o baml_src/01-agent.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/01-agent.baml && cat baml_src/01-agent.baml

Now let's create our main function that simulates command-line arguments:


In [ ]:
# ./walkthrough/01-main.py
import sys

def main():
    # Set default args if none provided
    if len(sys.argv) < 2:
        sys.argv = ["notebook", "hello from the notebook!"]
    
    # Get command line arguments, skipping the first one (script name)
    args = sys.argv[1:]
    
    if len(args) == 0:
        print("Error: Please provide a message as a command line argument", file=sys.stderr)
        return
    
    # Join all arguments into a single message
    message = " ".join(args)
    
    # Create a new thread with the user's message as the initial event
    thread = Thread([{"type": "user_input", "data": message}])
    
    # Run the agent loop with the thread
    result = agent_loop(thread)
    print(result)

Let's test our agent! You can modify the sys.argv line in the cell above to send different messages.


In [ ]:
baml_generate()

In [ ]:
main()